In [1]:
### Initial design of drawing a coplanar waveguide resonator 


# automatic reloading of modules when they change

%load_ext autoreload
%autoreload 2


In [2]:
# Import statements

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

In [3]:
# Instantiate QDesign class and initialise user interface

design = designs.DesignPlanar()
gui = MetalGUI(design)

In [4]:
# Allow overwriting

design.overwrite_enabled = True

In [54]:
otg = OpenToGround(design, 'open_to_ground', options=dict(pos_x='0um', pos_y='0.9mm',
orientation='180'))
stg = ShortToGround(design, 'short_to_ground', options=dict(pos_x='0um', pos_y='-1.25mm',
orientation='270')) 

from collections import OrderedDict
jogs = OrderedDict()
jogs[0] = ["L", '125um']
#jogs[1] = ["L", '500um']
jogs[1] = ["R", '100um']
#jogs[3] = ["R", '125um']

rt_meander = RouteMeander(design, 'readout', Dict(
        total_length='4 mm',
        hfss_wire_bonds = True,
        #fillet='90 um',
        lead = Dict(
        start_straight='0.2mm',
        end_straight='0mm',
        end_jogged_extension=jogs),
        pin_inputs=Dict(
            start_pin=Dict(component='short_to_ground', pin='short'),
            end_pin=Dict(component='open_to_ground', pin='open')), ))

gui.rebuild()
gui.autoscale()


In [10]:
import qiskit_metal
qiskit_metal.open_docs()